<a href="https://colab.research.google.com/github/SamuelK87/Machine-vision-based-defect-detection-in-welding-process/blob/master/model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import the necessary packages

In [3]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import imutils
from imutils import paths
import pickle
import cv2
from google.colab.patches import cv2_imshow
import os
import random

loading the image data

In [4]:
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 400
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

# initialize the data and labels
data = []
labels = []

# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images('/content/drive/My Drive/Colab Notebooks/weld_dataset_10cls')))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)
    # extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
    data.nbytes / (1024 * 1000.0)))

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
                                                  labels, test_size=0.2, random_state=42)

[INFO] loading images...
[INFO] data matrix: 194.62MB


load the trained convolutional neural network and the label
binarizer

In [5]:
print("[INFO] loading network...")
model = load_model('/content/drive/My Drive/Colab Notebooks/Results/wdef_10cls_400ep/Wld_10cls.model')
lb = pickle.loads(open('/content/drive/My Drive/Colab Notebooks/Results/wdef_10cls_400ep/lb.pickle', "rb").read())

[INFO] loading network...


making prediction and taking the classification report

In [10]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

[INFO] evaluating network...


AxisError: ignored